In [58]:
import pandas as pd
import xmlrpc.client
import pandas as pd

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)


# TIEMPOS ESTANDAR

In [2]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


tiempo_estimado_estandard = models.execute_kw(
    db, uid, password,
    'mrp.routing.workcenter', 'search_read',
    [[]],   # Solo cerradas
    {'fields': ['routing_id','total_nbr_minimo','display_name','workcenter_id']}
)

test = pd.DataFrame(tiempo_estimado_estandard)

test['producto'] = test['routing_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

test['centro_produccion'] = test['workcenter_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)


test['nom_produccion'] = test['producto'].str.extract(r"^(\S+)")

ruta = "/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx"
pareto_prod = pd.read_excel(ruta)

pareto_prod["producto"] = (
    pareto_prod["product_name"]
    .str.split("]").str[-1]   # toma lo que viene después de ']'
    .str.strip()              # quita espacios en blanco iniciales
)



pareto_prod['nom_produccion'] = pareto_prod['producto'].str.extract(r"^(\S+)")

# Merge claficacion

test = test.rename(columns={"total_nbr_minimo":"tiempo_estimado_estandard" , "display_name":"maquina_esperada"})

pareto_prod = pareto_prod[["nom_produccion","Clasificacion"]] #Columna realmente requeridas
te_clas = test.merge(pareto_prod, how="left",on="nom_produccion") 

# Descartar productos sin ventas ya que no tienen clasificacion
sin_ventas = te_clas[te_clas["Clasificacion"].isna()] 
te_clas = te_clas[~te_clas["nom_produccion"].isin(sin_ventas["nom_produccion"])]

te_clas = te_clas[["nom_produccion","producto","Clasificacion","centro_produccion","tiempo_estimado_estandard","maquina_esperada"]]


In [3]:
te_clas[te_clas["producto"].str.contains("DA4772A")]

,nom_produccion,producto,Clasificacion,centro_produccion,tiempo_estimado_estandard,maquina_esperada
125,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,CORTE,0.0,DESARROLLO Y ALTURA
563,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,CORTE,0.0,ENTUBADORA 4
2306,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,INTERNOS GUATA,0.0,GUATA
2726,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,INTERNOS GUATA,0.0,TERMOSELLADORA
3026,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,INYECCION,0.0,OMS
3631,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,EMBALAJE,0.0,BOLSA
4520,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,EMBALAJE,0.0,MARCADO/HOTMELT/ARANDELA/EMPAQUE
5278,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,EMBALAJE,0.0,CONTROL DE CALIDAD
5599,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,EMBALAJE,0.0,EMBALAJE


# INFO OP

In [72]:
import xmlrpc.client
import pandas as pd
import re

# --- Configuración de conexión ---
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- 1️⃣ Órdenes de producción terminadas ---
producciones_done = models.execute_kw(
    db, uid, password,
    "mrp.production", "search_read",
    [[("state", "=", "done")], ["id", "name"]]
)
df_producciones = pd.DataFrame(producciones_done)
ids_done = df_producciones["id"].tolist()

# --- 2️⃣ Work orders asociadas ---
workorders = models.execute_kw(
    db, uid, password,
    "mrp.production.workcenter.line", "search_read",
    [
        [("production_id", "in", ids_done)],
        [
            "name", "production_id", "product_id", "product_internal_ref",
            "workcenter_id", "date_start", "date_finished", "hour",
            "tiempo_segundos_hombre", "qty", "user_start", "fuerza_laboral",
            "state", "delay", "tiempo_estimado_x_fuerza_lab",
            "tiempo_estimado_segundos","tiempo_real_segundos"
        ]
    ]
)

df_workorders = pd.DataFrame(workorders)

import re

def extraer_nom_produccion_desde_name(valor):
    """Extrae el código del producto (ej: DA4570A) desde la columna 'name'."""
    if not isinstance(valor, str):
        return None
    match = re.search(r'\bDA\d{3,5}[A-Z]?\b', valor)
    return match.group(0) if match else None

df_workorders["nom_produccion"] = df_workorders["name"].apply(extraer_nom_produccion_desde_name)


# --- 4️⃣ Limpieza y creación de columnas ---
df_workorders["product_id_name"] = df_workorders["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["product_id_int"] = df_workorders["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None
)


df_workorders["OP"] = df_workorders["production_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["production_id_int"] = df_workorders["production_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None
)
df_workorders["operario"] = df_workorders["user_start"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["centro_produccion"] = df_workorders["workcenter_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["maquina_real"] = df_workorders["name"].str.split(" - ", n=1).str[0].str.strip()

df_workorders["nom_produccion"] = df_workorders["nom_produccion"].fillna("")


#TARDA 2 min

In [73]:
df_ops = df_workorders[["nom_produccion","OP","date_finished","centro_produccion","operario","qty","maquina_real","tiempo_segundos_hombre","delay","tiempo_estimado_x_fuerza_lab","tiempo_estimado_segundos","tiempo_real_segundos"]]

KeyError: "['tiempo_real_segundos'] not in index"

In [6]:
df_ops[df_ops["nom_produccion"].str.contains("4570", na=False)].sample(10)

,nom_produccion,OP,date_finished,centro_produccion,operario,qty,maquina_real,tiempo_segundos_hombre,delay,tiempo_estimado_x_fuerza_lab,tiempo_estimado_segundos
33978,DA4570,MO04771,2020-02-03 12:12:07,EMBALAJE,LEYDI PAOLA REYES RUBIO,801.0,MARCADO Y HOTMELT,156113.82,277298.0,0.00,0.0000
48635,DA4570A,MO04400,2019-10-31 12:34:09,EMBALAJE,MARIA ALEJANDRA ORDOÑEZ AVILA,207.0,EMBALAJE,4992.86,707485.0,0.00,0.0000
11921,DA4570,MO06942,2021-10-26 09:31:39,CORTE,NEL ALEJANDRO CHAPARRO LOTA,500.0,DESPUNTE Y ENROLLADO,20000.00,80089.0,20000.00,20000.0000
45569,DA4570A,MO09886,2023-11-18 10:41:33,EMBALAJE,OLGA YANETH TORRES TRIANA,300.0,CONTROL DE CALIDAD,10893.75,167412.0,4539.06,13617.1875
5142,DA4570,MO07640,2022-05-27 14:27:19,CORTE,BRAYAN NAIR MUÑOZ SALAS,500.0,DESARROLLO Y ALTURA,0.00,2.0,0.00,0.0000
18044,DA4570,MO07333,2022-02-15 07:52:28,CORTE,BLANCA DORA CUBIDES SANCHEZ,600.0,SOLDADURA,33230.00,242814.0,41537.50,41537.5000
48614,DA4570,MO04374,2019-10-23 14:08:33,EMBALAJE,LEYDI PAOLA REYES RUBIO,603.0,EMBALAJE,16642.65,114776.0,0.00,0.0000
38683,DA4570,MO10607,2024-05-23 09:08:30,EMBALAJE,ERIKA YULIEDT UMAÑA SABOGAL,600.0,MARCADO/HOTMELT/ARANDELA/EMPAQUE,9062.00,7896.0,11327.50,11327.5000
31114,DA4570,MO09700,2023-10-17 13:29:52,INYECCION,YISETH MARGARITA JULIO PEÑARANDA,600.0,INYECCION MESA INTERNOS,117609.50,363717.0,73505.94,147011.8750
39653,DA4570,MO11821,2025-01-30 08:27:01,EMBALAJE,ESTEFANNY ALVAREZ VILLANUEVA,600.0,MARCADO/HOTMELT/ARANDELA/EMPAQUE,16012.50,237332.0,5663.75,11327.5000


# TIEMPO OPERATIVO

In [66]:
import xmlrpc.client
import pandas as pd
from datetime import datetime

# ===============================
# 1️⃣ Conexión con Odoo 8
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# ===============================
# 2️⃣ Modelo a consultar
# ===============================
modelo = 'mrp.production.workcenter.line'  # cámbialo según necesites

# ===============================
# 3️⃣ Filtro: registros con fecha > 2023-01-01
# ===============================
fecha_inicio = '2023-01-01 00:00:00'
filtro = [('create_date', '>=', fecha_inicio)]

# ===============================
# 4️⃣ Consulta
# ===============================
records = models.execute_kw(
    db, uid, password,
    modelo, 'search_read',
    [filtro],
    {'limit': 1000000}
)

df_sample = pd.DataFrame(records)


In [74]:
df_sample["OP"] = df_sample["production_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)

#date_start inicio
#
df_sample[df_sample["OP"]=="MO11970"].head(1)

,origin,message_follower_ids,fecha_inicial_pause,create_date,es_contabilizado,sequence,production_state,tiempo_segundos_hombre,qty,fecha_final_pausa,tiempo_estimado_segundos,message_ids,user_finish,message_summary,create_uid,display_name,message_is_follower,__last_update,delay_pausa,date_start,message_last_post,state,id,delay,user_start,workcenter_id,uom,product,product_internal_ref,date_finished,date_planned_end,tiempo_optimo_segundos,tiempo_optimo_horas,write_date,tiempo_estimado_x_fuerza_lab,write_uid,cycle,message_unread,name,hour,date_planned,cantidad_hombres,product_id,production_id,website_message_ids,fuerza_laboral,OP
974,OP22502105,[11156],False,2025-02-15 15:41:39,True,0,done,5668.91,60.0,False,7086.136364,"[11477803, 11477802]","[292, ALVARO MELO PRADA]",,"[369, BRAYAN NAIR MUÑOZ SALAS]",TAPAS - DA4133 FILTRO AIRE 2_ JOHN DEERE,False,2025-02-27 15:05:54,0.0,2025-02-18 08:59:22,False,done,68288,781595.0,"[292, ALVARO MELO PRADA]","[54, CORTE]","[1, Und(es)]","[17492, [DAE04133025] DA4133 FILTRO AIRE 2_ JOHN DEERE (025 DA4133)]",DA4133TE,2025-02-27 10:05:57,2025-02-20 07:58:12,2091.0,0.580833,2025-02-27 15:05:54,7086.14,"[723, SIOMARA HERNANDEZ]",60.0,False,TAPAS - DA4133 FILTRO AIRE 2_ JOHN DEERE,1.97,2025-02-20 00:00:00,1.0,"[17492, [DAE04133025] DA4133 FILTRO AIRE 2_ JOHN DEERE (025 DA4133)]","[12160, MO11970]",[],[94781],MO11970


# AGRUPACION

In [26]:
import pandas as pd

# Copiamos el dataset original
df = df_aaa.copy()
df["date_finished"] = pd.to_datetime(df["date_finished"], errors="coerce")

# 🔹 Filtrar fechas desde 2023
df = df[df["date_finished"] >= pd.Timestamp("2023-01-01")].copy()

# 🔹 Determinar fila representativa (máquina esperada o mayor estándar)
cols_base = ["nom_produccion", "OP", "date_finished", "operario"]
df["match_maquina"] = df["maquina_real"] == df["maquina_esperada"]

def seleccionar_fila(grupo):
    coincidentes = grupo[grupo["match_maquina"]]
    if not coincidentes.empty:
        return coincidentes.iloc[0]
    grupo_pos = grupo[grupo["tiempo_estimado_estandard"] > 0]
    if not grupo_pos.empty:
        return grupo_pos.sort_values("tiempo_estimado_estandard", ascending=False).iloc[0]
    return grupo.iloc[0]

df_filtrado = (
    df.groupby(cols_base, group_keys=False)
    .apply(seleccionar_fila)
    .reset_index(drop=True)
)

# 🔹 Calcular tiempo estándar total por producto (solo 1 valor por nom_produccion)
tiempos_estandar = (
    df_filtrado.groupby("nom_produccion", as_index=False)
    .agg({"tiempo_estimado_estandard": "max"})  # usamos MAX para dejarlo fijo
    .rename(columns={"tiempo_estimado_estandard": "tiempo_estandar_fijo"})
)

# 🔹 Consolidar producción
cols_group = [
    "nom_produccion", "OP", "date_finished",
    "centro_produccion", "maquina_real",
    "Clasificacion", "producto", "maquina_esperada", "tasa_prod_real"
]

df_final = (
    df_filtrado.groupby(cols_group, as_index=False)
    .agg({
        "qty": "sum",
        "tiempo_segundos_hombre": "sum",
        "operario": lambda x: ", ".join(sorted(x.unique()))
    })
)

# 🔹 Asociar el tiempo estándar fijo a cada producto
df_final = df_final.merge(tiempos_estandar, on="nom_produccion", how="left")

# 🔹 Asegurarse que no se repita en PBI
df_final["tiempo_estimado_estandard"] = df_final["tiempo_estandar_fijo"]

# 🔹 (opcional) Calcular eficiencia
df_final["eficiencia_%"] = (
    df_final["tasa_prod_real"] / df_final["tiempo_estandar_fijo"] * 100
).round(2)

print(df_final[["nom_produccion", "OP", "tiempo_estimado_estandard"]].drop_duplicates())




/tmp/ipykernel_207598/2391601235.py:25: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(seleccionar_fila)


     nom_produccion       OP  tiempo_estimado_estandard
0            DA2020  MO08000                      485.0
3            DA2020  MO08186                      485.0
8            DA2020  MO08303                      485.0
13           DA2020  MO08304                      485.0
18           DA2020  MO08305                      485.0
...             ...      ...                        ...
4996         DA8168  MO11847                      150.0
5003         DA8168  MO11960                      150.0
5010         DA8168  MO12269                      150.0
5017         DA8168  MO12514                      150.0
5026         DA8168  MO12837                      150.0

[699 rows x 3 columns]


In [27]:
df_final.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/tiempo_estimado_estandard.xlsx")